# Python Object Oriented Programming

Author: Sean McLaughlin

Object Oriented Programming (OOP) is arguably the most popular programming paradigm. It refers to a "style" of programming, partly in the user's control and partly emphasized by the programming language itself. Python, for example, was built from the start as an OOP language. That means that much of the language's built-ins follow and OOP design, and making your own objects is easy. 

**Disclaimer:** OOP is a powerful and popular programming paradigm. It is not, however, the only one. Some people have very, *very* strong opinions on this. For the next hour, we'll ignore those people. However, keep in mind going forward there are other was to design a program, and OOP is not always the best way! 

# TODO DELETME
Topics to cover:
* Make a class
    * Instance variables
    * Instance methods
    * Difference between instances and the class itself
* New v. Old style classes?
* Iheritance
* Operator overloading
* Abstract classes

-----
### Part 0: What is OOP?

What *is* an object?

Well, first what *is* programming? 

Programming, in the most abstract sense, is defining logic to perform manipulations on data. 

An object is an abstract... um... *object* that contains both its relevant data and functions to manipulate that data. It follows from the idea that what we really care about are these abstract objects than the logic or data separately. 

Ok sure but... what *is* an object?

It can be a lot of things. It can represent something real, like a student or an astronomical object. It can represent something more abstract, like a statistical model or a cosmology. It could even be something completely abstract. For example, all the widgets and cells in this notebook are actually objects within the Jupyter code. 


Programming with objects allows you to design your programs the way you actually understand the logic yourself. 

---
### Part 1: Python Classes 

Let's make a simple class to get started. 

In [7]:
class Student(object):
    'A class defining a student!'
    
    def __init__(self, name, gpa, major = 'Physics'):
        '''
        Initialize the student. 
        
        name: String, the student's name
        gpa: The student's overall gpa
        major: The student's major. Default is Physics. 
        '''
        self.name = name
        self.gpa = gpa
        self.major = major
        
    def change_major(self, new_major):
        '''
        Change the student's major. 
        
        new_major: str, the student's new major
        '''
        if new_major != self.major:
            self.major=new_major
        else:
            print "%s is already %s's major!"%(self.major, self.name)
        

There's a lot to unpack here, so lets break it down. We've defined a class `Student` that has three attributes and two methods.

The attributes are:
* name
* gpa
* major

And the methods are:
* \__init__
* change_major

How do these work? Let's make some instances to see. 

In [8]:
student1 = Student('Sean', 3.0)
print student1.name, student1.gpa, student1.major
student2 = Student('Alice', 4.5, major='Biology')

student2.change_major('Physics')#she wised up
student1.change_major('Physics')#this should print! 

Sean 3.0 Physics
Physics is already Sean's major!


There's a few things to notice here:
* student1 and student2 are *instances* of the student object. 
* We can access a Student's attributes and methods via `.` syntax

In [ ]:
from math import pi
class Planet(object):
    'A class defining the attributes of a planet.'
    
    G = 6.67e-11 #Newton's constant in SI units
    
    def __init__(self,name, order, mass,radius, moons=[]):
        '''
        Initialize the planet object.
        name: str, the name of the planet
        order: int, the planet's order in distance from the sun. 
        mass: float, The mass of the planet (in kg)
        radius: float, radius of the planet (in m)
        moons: list, a list of the names of the planet's moons. Default is an empty list. 
        '''
        self.name
        self.order = order
        self.mass = mass
        self.radius = radius
        self.moons=moons
        
        volume = 4/3*pi*self.radius**3
        self.density = self.mass/volume
        
    def __cmp__(self, other):
        '''
        Define comparison between planets. Defined as the planet closer to the sun is "less"
        
        other: Another Planet object
        
        return: the difference of self and other's orders. 
        '''
        return self.order-other.order
        
    def num_moons(self):
        '''
        Return the number of moons the planet has
        
        return: Int, the number of moons
        '''
        return len(self.moons)
    
    def gravity(self, r):
        '''
        Calculate the acceleration due to gravity of the planet at a distance r.
        r: float, the distance from the planet's center to calculate the force of gravity.
        
        return: float, the strength of gravity in m/s^2
        '''
        return G*self.mass/(r*r)
    
    def surface_gravity(self):
        '''
        Calculate the acceleration due to gravity on the planet's surface. 
        
        return: float, the surface gravity in m/s^2
        '''
        return self.gravity(self.radius)